In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='shap')

# import classes
from Tools import DateTimeSeriesSplit, Kraken

# model and metric for regression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE


In [2]:
# --- НОВАЯ МЕТРИКА ---
def median_median_ape(y_true, y_pred, group_code):
    """
    Рассчитывает Медиану Медианной Абсолютной Процентной Ошибки (MdMdAPE).
    Игнорирует group_code, если он None.

    Args:
        y_true: Series или array-like с фактическими значениями.
        y_pred: Series или array-like с предсказанными значениями.
        group_code: Series или array-like с кодами групп для агрегации (может быть None).

    Returns:
        Значение метрики Median of Median APE, или NaN, если расчет не удался.
    """
    if y_true is None or y_pred is None: return np.nan
    # Преобразуем входы в pandas Series для удобства
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_pred = pd.Series(y_pred).reset_index(drop=True)

    # Создаем DataFrame
    df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})

    # Рассчитываем APE
    df['ape'] = np.abs(df['y_pred'] - df['y_true']) / np.maximum(np.abs(df['y_true']), 1) * 100

    # Если group_code не предоставлен, считаем просто медиану APE по всем данным
    if group_code is None:
        metric = df['ape'].median(skipna=True)
    else:
        group_code = pd.Series(group_code).reset_index(drop=True)
        if len(group_code) != len(df):
             print("[!] median_median_ape: Длина group_code не совпадает с y_true/y_pred!")
             return np.nan # Возвращаем NaN если размеры не совпадают
        df['group_code'] = group_code
        # Рассчитываем медиану APE для каждой группы
        median_ape_per_group = df.groupby('group_code')['ape'].median()
        # Рассчитываем медиану от медиан по группам
        metric = median_ape_per_group.median(skipna=True)

    return metric

In [3]:
# set random seed value
seed_value = 23
np.random.seed(seed_value)

# set random seed value
seed_value = 23
np.random.seed(seed_value)

# creating dataset
c = 12052
X1 = pd.DataFrame()
X1['var_1'] = np.random.rand(c)
X1['var_2'] = np.random.rand(c)
X1['var_3'] = np.random.rand(c)
X1['var_4'] = np.random.rand(c)
X1['var_5'] = np.random.rand(c)
X1['var_6'] = np.random.rand(c)
X1['var_7'] = np.random.rand(c)
X1['var_8'] = np.random.rand(c)
X1['var_9'] = np.random.rand(c)
X1['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# create simple dependency for first part of data
y1 = X1['var_1'] + 3 * X1['var_2'] - np.power(X1['var_3'], 1.5)

X2 = pd.DataFrame()
X2['var_1'] = np.random.rand(c)
X2['var_2'] = np.random.rand(c)
X2['var_3'] = np.random.rand(c)
X2['var_4'] = np.random.rand(c)
X2['var_5'] = np.random.rand(c)
X2['var_6'] = np.random.rand(c)
X2['var_7'] = np.random.rand(c)
X2['var_8'] = np.random.rand(c)
X2['var_9'] = np.random.rand(c)
X2['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# add var_4 to dependency for second part
y2 = X2['var_1'] + 3 * X2['var_2'] - np.power(X2['var_3'], 1.5) + 2 * X2['var_4']

X3 = pd.DataFrame()
X3['var_1'] = np.random.rand(c)
X3['var_2'] = np.random.rand(c)
X3['var_3'] = np.random.rand(c)
X3['var_4'] = np.random.rand(c)
X3['var_5'] = np.random.rand(c) 
X3['var_6'] = np.random.rand(c)
X3['var_7'] = np.random.rand(c)
X3['var_8'] = np.random.rand(c)
X3['var_9'] = np.random.rand(c)
X3['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# add var_4 and var_5 to dependency for third part
y3 = X3['var_1'] + 3 * X3['var_2'] - np.power(X3['var_3'], 1.5) + 2 * X3['var_4'] + 4 * X3['var_5']

X = pd.concat([X1, X2, X3], axis=0)
y = pd.concat([y1, y2, y3], axis=0)

print("Regression dataset shape:", X.shape)

# --- СОЗДАЕМ GROUP_CODE (например, по году) ---
X['group_code_year'] = X['date'].dt.year
group_code_for_metric = X['group_code_year']
# ---

# creating cross validator
cv_datetime = DateTimeSeriesSplit(window=3000, n_splits=3, test_size=300, margin=0)
group_dt = X['date'] # Для DateTimeSeriesSplit

Regression dataset shape: (36156, 10)


In [4]:
# create model for selector
model_reg = LGBMRegressor(max_depth=3, verbosity=-1)

# create list of variables for model
list_of_vars = list(X.columns)
list_of_vars.remove('date')
list_of_vars.remove('group_code_year') # <-- Удаляем новый столбец из списка фичей
if 'index_time' in list_of_vars:
    list_of_vars.remove('index_time')

# create selector
selector_reg = Kraken(
    estimator=model_reg,
    cv=cv_datetime,
    # metric=MAPE, # <-- Заменяем метрику
    metric=median_median_ape, # <-- Новая метрика
    meta_info_name='example_regression_mdmdape', # <-- Меняем имя для лога
    task_type='regression',
    greater_is_better=False, # <-- MdMdAPE - чем меньше, тем лучше
    comparison_precision= 4 # <-- Увеличим точность для MdMdAPE
)

# get rank dictionary from variables
selector_reg.get_rank_dict(
    X=X[list_of_vars], # <-- Передаем X без 'date' и 'group_code'
    y=y,
    list_of_vars=list_of_vars,
    group_dt=group_dt,
    group_code=group_code_for_metric # <-- Передаем group_code
)

print("Rank dict (regression, MdMdAPE) top-5:", dict(list(selector_reg.rank_dict.items())[:5]))

# get variables
best_vars_reg = selector_reg.get_vars(
    X=X[list_of_vars], # <-- Передаем X без 'date' и 'group_code'
    y=y,
    rank_dict=selector_reg.rank_dict,
    group_dt=group_dt,
    group_code=group_code_for_metric, # <-- Передаем group_code
    max_feature_search_rounds=10
)
print("Selected vars (regression, MdMdAPE):", best_vars_reg)

[get_rank_dict] Starting combined baseline evaluation and SHAP calculation...
Fold: 1/3 | Status: Done (0.22s)              | Fold Time:   0.22s | Total Time:    0.25s                              
Fold: 2/3 | Status: Done (0.19s)              | Fold Time:   0.19s | Total Time:    0.44s                              
Fold: 3/3 | Status: Done (0.18s)              | Fold Time:   0.18s | Total Time:    0.63s                              
------------------------------
[get_rank_dict] >> FINAL Baseline Performance (All Features)
    Mean CV Score: 37.9794
    Fold Scores: [38.6683 37.2442 38.0256]
[get_rank_dict] Completed calculation. Total time: 0.69 seconds.
Rank dict (regression, MdMdAPE) top-5: {'var_2': 1, 'var_5': 2, 'var_4': 3, 'var_3': 4, 'var_1': 5}
[get_vars] Evaluating initial feature set (if any)...
[get_vars] Starting feature selection procedure...
[get_vars] Starting from scratch (will check top 10 features first).

--- Starting Step: Selecting feature #1 (Checking top 9) ---

In [8]:
# create model for selector
model_reg = LGBMRegressor(max_depth=3, verbosity=-1)

# create list of variables for model
list_of_vars = list(X.columns)
list_of_vars.remove('date')
list_of_vars.remove('group_code_year') # <-- Удаляем новый столбец из списка фичей
if 'index_time' in list_of_vars:
    list_of_vars.remove('index_time')

# Рассчитаем смещение для избежания отрицательных значений при преобразовании
min_y = np.min(y)
shift_value = abs(min_y) if min_y < 0 else 0
y_corrected = y + shift_value  # Корректируем таргет

# create selector with SAFE target transformation
selector_reg = Kraken(
    estimator=model_reg,
    cv=cv_datetime,
    metric=median_median_ape,
    meta_info_name='example_regression_mdmdape',
    task_type='regression',
    greater_is_better=False,
    comparison_precision=4,
    forward_transform=lambda x: np.sqrt(x + shift_value),  # Безопасное преобразование
    inverse_transform=lambda x: np.square(x) - shift_value  # Корректное обратное преобразование
)

# get rank dictionary from variables
selector_reg.get_rank_dict(
    X=X[list_of_vars],
    y=y_corrected,  # Используем скорректированный таргет
    list_of_vars=list_of_vars,
    group_dt=group_dt,
    group_code=group_code_for_metric
)

print("Rank dict (regression, MdMdAPE) top-5:", dict(list(selector_reg.rank_dict.items())[:5]))

# get variables
best_vars_reg = selector_reg.get_vars(
    X=X[list_of_vars],
    y=y_corrected,  # Используем скорректированный таргет
    rank_dict=selector_reg.rank_dict,
    group_dt=group_dt,
    group_code=group_code_for_metric,
    max_feature_search_rounds=10
)
print("Selected vars (regression, MdMdAPE):", best_vars_reg)

[get_rank_dict] Starting combined baseline evaluation and SHAP calculation...
Fold: 1/3 | Status: Done (0.18s)              | Fold Time:   0.18s | Total Time:    0.21s                              
Fold: 2/3 | Status: Done (0.19s)              | Fold Time:   0.19s | Total Time:    0.41s                              
Fold: 3/3 | Status: Done (0.19s)              | Fold Time:   0.19s | Total Time:    0.60s                              
------------------------------
[get_rank_dict] >> FINAL Baseline Performance (All Features)
    Mean CV Score: 30.1081
    Fold Scores: [30.6971 30.1135 29.5136]
[get_rank_dict] Completed calculation. Total time: 0.66 seconds.
Rank dict (regression, MdMdAPE) top-5: {'var_2': 1, 'var_4': 2, 'var_5': 3, 'var_3': 4, 'var_1': 5}
[get_vars] Evaluating initial feature set (if any)...
[get_vars] Starting feature selection procedure...
[get_vars] Starting from scratch (will check top 10 features first).

--- Starting Step: Selecting feature #1 (Checking top 9) ---

In [5]:
# Пробуем CatBoostRegressor с квантильной регрессией для MdMdAPE
# Квантильная регрессия лучше подходит для метрик, связанных с медианами

# Создаем CatBoost модель с квантильной функцией потерь
from catboost import CatBoostRegressor
catboost_quantile = CatBoostRegressor(
    iterations=1000,  # Увеличиваем количество итераций
    learning_rate=0.1,
    depth=4,
    loss_function='Quantile:alpha=0.5',  # Правильный формат для квантильной регрессии
    verbose=0,
    random_seed=42
)

# Инициализируем селектор для квантильной регрессии
selector_catboost_quantile = Kraken(
    estimator=catboost_quantile,
    cv=cv_datetime,
    metric=median_median_ape,
    meta_info_name='example_quantile_catboost_mdmdape',
    task_type='regression',
    greater_is_better=False,
    comparison_precision=4
)

# Получаем ранжирование фичей
selector_catboost_quantile.get_rank_dict(
    X=X[list_of_vars],
    y=y,
    list_of_vars=list_of_vars,
    group_dt=group_dt,
    group_code=group_code_for_metric
)

print("Rank dict (Quantile CatBoost, MdMdAPE) top-5:", dict(list(selector_catboost_quantile.rank_dict.items())[:5]))

# Отбираем лучшие фичи
best_vars_catboost_quantile = selector_catboost_quantile.get_vars(
    X=X[list_of_vars],
    y=y,
    rank_dict=selector_catboost_quantile.rank_dict,
    group_dt=group_dt,
    group_code=group_code_for_metric,
    max_feature_search_rounds=10
)
print("Selected vars (Quantile CatBoost, MdMdAPE):", best_vars_catboost_quantile)


[get_rank_dict] Starting combined baseline evaluation and SHAP calculation...
Fold: 1/3 | Status: Done (1.45s)              | Fold Time:   1.45s | Total Time:    1.48s                              
Fold: 2/3 | Status: Done (1.35s)              | Fold Time:   1.35s | Total Time:    2.83s                              
Fold: 3/3 | Status: Done (1.32s)              | Fold Time:   1.32s | Total Time:    4.15s                              
------------------------------
[get_rank_dict] >> FINAL Baseline Performance (All Features)
    Mean CV Score: 30.2660
    Fold Scores: [30.181  30.2174 30.3997]
[get_rank_dict] Completed calculation. Total time: 4.21 seconds.
Rank dict (Quantile CatBoost, MdMdAPE) top-5: {'var_2': 1, 'var_4': 2, 'var_3': 3, 'var_1': 4, 'var_5': 5}
[get_vars] Evaluating initial feature set (if any)...
[get_vars] Starting feature selection procedure...
[get_vars] Starting from scratch (will check top 10 features first).

--- Starting Step: Selecting feature #1 (Checking top

In [6]:
# Определяем функции трансформации
def forward_transform(y):
    # Добавляем небольшую константу для стабильности, если в y есть нули или очень малые значения
    return np.log1p(y)

def inverse_transform(y_transformed):
    return np.expm1(y_transformed)


model_reg_quantile_log = LGBMRegressor(
    objective='quantile',
    alpha=0.5,
    max_depth=3,
    verbosity=-1,
    random_state=42
)

selector_reg_quantile_log = Kraken(
    estimator=model_reg_quantile_log,
    cv=cv_datetime,
    metric=median_median_ape, # Метрика будет считаться на оригинальной шкале
    meta_info_name='example_quantile_log_regression_mdmdape', # Новое имя для логов
    #forward_transform=forward_transform, # Прямое преобразование таргет
    #inverse_transform=inverse_transform, # Обратное преобразование для метрики
    task_type='regression',
    greater_is_better=False,
    comparison_precision= 4
)

# --- Выполнение отбора ---

# get rank dictionary from variables
print("\nCalculating rank dict for Quantile Regression (alpha=0.5)")
selector_reg_quantile_log.get_rank_dict(
    X=X[list_of_vars].copy(), # Передаем копию, чтобы избежать изменений в оригинальном X
    y=y.copy(),             # Передаем копию
    list_of_vars=list_of_vars,
    group_dt=group_dt,
    group_code=group_code_for_metric
)

if selector_reg_quantile_log.rank_dict:
    print("Rank dict (Quantile Log, MdMdAPE) top-5:", dict(list(selector_reg_quantile_log.rank_dict.items())[:5]))
else:
    print("Rank dict calculation failed or returned empty.")

# get variables (только если rank_dict не пустой)
if selector_reg_quantile_log.rank_dict:
    print("\nSelecting features for Quantile Regression (alpha=0.5) with log-transformed target...")
    best_vars_reg_quantile_log = selector_reg_quantile_log.get_vars(
        X=X[list_of_vars].copy(), # Передаем копию
        y=y.copy(),             # Передаем копию
        rank_dict=selector_reg_quantile_log.rank_dict,
        group_dt=group_dt,
        group_code=group_code_for_metric,
        max_feature_search_rounds=10
    )
    print("Selected vars (Quantile Log, MdMdAPE):", best_vars_reg_quantile_log)
else:
    print("Skipping feature selection as rank_dict is empty.")
    best_vars_reg_quantile_log = []


Calculating rank dict for Quantile Regression (alpha=0.5) with log-transformed target...
[get_rank_dict] Starting combined baseline evaluation and SHAP calculation...
Fold: 1/3 | Status: Done (0.18s)              | Fold Time:   0.18s | Total Time:    0.21s                              
Fold: 2/3 | Status: Done (0.19s)              | Fold Time:   0.19s | Total Time:    0.39s                              
Fold: 3/3 | Status: Done (0.18s)              | Fold Time:   0.18s | Total Time:    0.58s                              
------------------------------
[get_rank_dict] >> FINAL Baseline Performance (All Features)
    Mean CV Score: 31.0837
    Fold Scores: [30.287  31.6209 31.3433]
[get_rank_dict] Completed calculation. Total time: 0.63 seconds.
Rank dict (Quantile Log, MdMdAPE) top-5: {'var_2': 1, 'var_4': 2, 'var_3': 3, 'var_1': 4, 'var_5': 5}

Selecting features for Quantile Regression (alpha=0.5) with log-transformed target...
[get_vars] Evaluating initial feature set (if any)...
[g